In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
OPENAI_API_KEY = config['OPENAI']['OPENAI_API_KEY']
YOUTUBE_KEY = config['YOUTUBE']['YOUTUBE_API_KEY']
NAVER_CLIENT_ID = config['NAVER']['NAVER_CLIENT_ID']
NAVER_CLIENT_SECRET = config['NAVER']['NAVER_CLIENT_SECRET']
GOOGLE_SEARCH_KEY = config['GOOGLE']['GOOGLE_API_KEY']
CSE_ID = config['GOOGLE']['CSE_ID']
SERPAPI_API_KEY = config['SERPAPI']['SERPAPI_API_KEY']


os.environ.update({'OPENAI_API_KEY': OPENAI_API_KEY})
os.environ.update({'YOUTUBE_KEY': YOUTUBE_KEY})
os.environ.update({'NAVER_CLIENT_ID': NAVER_CLIENT_ID})
os.environ.update({'NAVER_CLIENT_SECRET': NAVER_CLIENT_SECRET})
os.environ.update({'GOOGLE_SEARCH_KEY': GOOGLE_SEARCH_KEY})
os.environ.update({'CSE_ID': CSE_ID})
os.environ.update({'SERPAPI_API_KEY': SERPAPI_API_KEY})

In [3]:
from langchain.prompts import load_prompt, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

In [35]:
qna_plan_template = '''
I want you to make a plan for answering the user.
Think step by step and write down the plan.
You can ask questions to the user to make a plan. 
Answer in korean.

If you want to ask to the user, use the following format:
<format>
User: user input.
Thought1: I need to ask to the user.
Ask to user: the question to ask to the user.
End of Plan
</format>
Don't make a Plan when you ask question to user.

If you can make the plan, Use the following format:
<format>
User: user input.
Thought1: the first thought.
Thought2: the second thought.
... (this Thought can repeat as many times as you want)
Step1: the first step.
Step2: the second step.
... (this Step can repeat as many times as you want)
End of Plan
</format>

Begin!
<format>
{chat_history}
User: {user_input}
Thought1: 
'''

In [3]:
from models.llm.chain import BaseChain
from typing import List

In [45]:
import re

class QnAPlanChain(BaseChain):
    def __init__(self, 
                qna_plan_template=None, 
                input_variables:List[str]=None,
                qna_plan_template_path='../openai_skt/models/templates/qna_plan_prompt.json', 
                model='gpt-4', 
                verbose=False) -> None:
        super().__init__(template=qna_plan_template, input_variables=input_variables, template_path=qna_plan_template_path, model=model, verbose=verbose)

    def run(self, user_input:str=None, chat_history:List[List[str]]=None):
        return super().run(user_input=user_input, chat_history=chat_history)
    
    async def arun(self, user_input:str=None, chat_history:List[List[str]]=None):
        return await super().arun(user_input=user_input, chat_history=chat_history)

    def parse_input(self, user_input:str=None, chat_history:List[List[str]]=None):
        chat_history_text = ""
        for chat in chat_history[-2:]:
            chat_history_text += f"User: {chat[0]}\n"
            chat_history_text += f"Agent: {chat[1]}\n"
        return {"chat_history": chat_history_text, "user_input": user_input}

    def parse_output(self, output:str=None):
        if 'Ask to user: ' in output:
            return output.split('Ask to user: ')[1].strip().split('\n')[0].strip()
        elif 'Step' in output:
            plans = re.findall(r'Step\d+: (?:\n)?(.*?\.)(?=\s|$)', output)
            return plans
        else:
            raise ValueError('Invalid output format')

In [46]:
qna_plan_chain = QnAPlanChain(qna_plan_template, input_variables=['user_input', 'chat_history'], verbose=True)

In [38]:
chat_history = []

In [50]:
user_input = '서울'
result = qna_plan_chain.run(user_input=user_input, chat_history=chat_history)



> Entering new LLMChain chain...
Prompt after formatting:

I want you to make a plan for answering the user.
Think step by step and write down the plan.
You can ask questions to the user to make a plan. 
Answer in korean.

If you want to ask to the user, use the following format:
<format>
User: user input.
Thought1: I need to ask to the user.
Ask to user: the question to ask to the user.
End of Plan
</format>
Don't make a Plan when you ask question to user.

If you can make the plan, Use the following format:
<format>
User: user input.
Thought1: the first thought.
Thought2: the second thought.
... (this Thought can repeat as many times as you want)
Step1: the first step.
Step2: the second step.
... (this Step can repeat as many times as you want)
End of Plan
</format>

Begin!
<format>
User: 오늘 날씨가 어때?
Agent: 현재 어느 지역에 계신가요?

User: 서울
Thought1: 




> Finished chain.
사용자가 서울에 있다고 했으므로, 서울의 날씨 정보를 찾아야 한다.
Thought2: 
날씨 정보는 실시간으로 변하기 때문에, 가장 최신의 정보를 제공해야 한다.

Step1: 
인터넷에서 서울의 최신 날씨 정보를 검색한다.
Step2: 
검색된 날씨 정보를 바탕으로 사용자에게 답변을 준비한다.
Step3: 
준비된 답변을 사용자에게 제공한다.
End of Plan


In [51]:
result

['인터넷에서 서울의 최신 날씨 정보를 검색한다.',
 '검색된 날씨 정보를 바탕으로 사용자에게 답변을 준비한다.',
 '준비된 답변을 사용자에게 제공한다.']

In [49]:
chat_history.append([user_input, result])

In [101]:
qna_critic_template = '''
You are the critic who determines if the current step is working.
The answer options are as follows:
<option>
A. The plan is working. Go to the next step.
B. The plan is not working. Perform this step again.
</option>

Use the following format:
<format>
User: user input.
Plan1: the first plan.
Obs1: obs1 is the result of executing against that plan1.
Plan2: the second plan.
Obs2: the second observation.
... (this Plan can repeat as many times as you want)
Final Thought: Thoughts on how the plan and its observations are going.
Final Answer: A or B
</format>

Begin!
<format>
{chat_history}
User: {user_input}
{plan_and_obs}
Final Thought: 
'''

In [116]:
class QnACriticChain(BaseChain):
    def __init__(self, 
                qna_critic_template=None, 
                input_variables:List[str]=None,
                qna_critic_template_path='../openai_skt/models/templates/qna_critic_prompt.json', 
                model='gpt-4', 
                verbose=False) -> None:
        super().__init__(template=qna_critic_template, input_variables=input_variables, template_path=qna_critic_template_path, model=model, verbose=verbose)

    def run(self, user_input:str=None, plan_and_obs:List=None, chat_history:List[List[str]]=None):
        return super().run(user_input=user_input, plan_and_obs=plan_and_obs, chat_history=chat_history)
    
    async def arun(self, user_input:str=None, plan_and_obs:List[List[str]]=None, chat_history:List[List[str]]=None):
        return await super().arun(user_input=user_input, plan_and_obs=plan_and_obs, chat_history=chat_history)

    def parse_input(self, user_input:str=None, plan_and_obs:List[List[str]]=None, chat_history:List[List[str]]=None):
        chat_history_text = ""
        for user, agent in chat_history[-3:]:
            chat_history_text += f"User: {user}\n"
            chat_history_text += f"Agent: {agent}\n"

        plan_and_obs_text = ""
        for idx, (plan, obs) in enumerate(plan_and_obs):
            plan_and_obs_text += f"Plan{idx+1}: {plan}\n"
            plan_and_obs_text += f"Obs{idx+1}: {obs}\n"
        return {"chat_history": chat_history_text, "user_input": user_input, "plan_and_obs": plan_and_obs_text}
    
    def parse_output(self, output:str=None):
        return output.split('Final Answer: ')[1][0]
        

In [117]:
qna_critic_chain = QnACriticChain(qna_critic_template, input_variables=['user_input', 'chat_history', 'plan_and_obs'], verbose=True)

In [118]:
plan_and_obs = [['인터넷에서 서울의 최신 날씨 정보를 검색한다.', '서울은 현재 25도 입니다.'], ['검색된 날씨 정보를 바탕으로 사용자에게 답변을 준비한다.', '서울의 현재 날씨는 21도 입니다.']]

In [119]:
result = qna_critic_chain.run(user_input=user_input, plan_and_obs=plan_and_obs, chat_history=chat_history)



> Entering new LLMChain chain...
Prompt after formatting:

You are the critic who determines if the current step is working.
The answer options are as follows:
<option>
A. The plan is working. Go to the next step.
B. The plan is not working. Perform this step again.
</option>

Use the following format:
<format>
User: user input.
Plan1: the first plan.
Obs1: obs1 is the result of executing against that plan1.
Plan2: the second plan.
Obs2: the second observation.
... (this Plan can repeat as many times as you want)
Final Thought: Thoughts on how the plan and its observations are going.
Final Answer: A or B
</format>

Begin!
<format>
User: 오늘 날씨가 어때?
Agent: 현재 어느 지역에 계신가요?

User: 서울
Plan1: 인터넷에서 서울의 최신 날씨 정보를 검색한다.
Obs1: 서울은 현재 25도 입니다.
Plan2: 검색된 날씨 정보를 바탕으로 사용자에게 답변을 준비한다.
Obs2: 서울의 현재 날씨는 21도 입니다.

Final Thought: 




> Finished chain.


In [120]:
result

'B'